In [ ]:
import requests
import json
import csv
import io

# ================= Configuration =================
URL = "http://localhost:3000/v1/chat/completions"
HEADERS = {"Content-Type": "application/json"}
MODEL_NAME = "your-model-name"  # Replace with the actual model name loaded in your server

# Define the 13 categorization rules (System Prompt)
CATEGORIES_PROMPT = """
You are an intelligent financial assistant. Classify the transaction into one of these categories:

1. 🏠 Housing & Home
2. 💡 Utilities
3. 🍔 Food & Dining
4. 🚗 Transportation
5. ❤️ Health & Wellness
6. 🛍️ Personal Spending & Shopping
7. 🎬 Entertainment
8. ✈️ Travel
9. 👶 Children & Dependents
10. 🐶 Pets
11. 🎁 Gifts & Donations
12. 💸 Financial & Debt
13. 📦 Miscellaneous & Other

Output valid JSON only.
Format:
{
  "category": "Category Name",
  "merchant_clean": "Cleaned Merchant Name",
  "reasoning": "Brief explanation"
}
"""

# ================= Data Source (CSV) =================
# Paste your raw CSV data here
csv_data = """Date,Time,Cardholder,Amount,Points,Balance,Status,Type,Merchant,Description
12/31/2024,4:13 PM,Lich,18.5,56,4705.84,Posted,Purchase,Canva Us Inc.,Canva US Inc. Kent DE"""

# ================= Function Definitions =================
def classify_transaction(row_dict):
    """
    Sends a single transaction row to the LLM for classification.
    """
    # Construct a description string that is easy for the model to understand
    transaction_desc = (
        f"Date: {row_dict['Date']}, "
        f"Merchant: {row_dict['Merchant']}, "
        f"Description: {row_dict['Description']}, "
        f"Amount: ${row_dict['Amount']}"
    )

    payload = {
        "model": MODEL_NAME,
        "max_tokens": 512,
        "temperature": 0.0,  # Set to 0.0 for the most stable/deterministic results
        "response_format": { "type": "json_object" },
        "messages": [
            {"role": "system", "content": CATEGORIES_PROMPT},
            {"role": "user", "content": f"Classify this transaction:\n{transaction_desc}"}
        ],
        "stream": False
    }

    try:
        response = requests.post(URL, headers=HEADERS, json=payload)
        response.raise_for_status()
        
        # Parse the returned JSON string content
        content = response.json()['choices'][0]['message']['content']
        return json.loads(content)
        
    except Exception as e:
        return {"error": str(e)}

# ================= Main Logic =================
def main():
    # Use io.StringIO to treat the string as a file object for the CSV reader
    f = io.StringIO(csv_data.strip())
    reader = csv.DictReader(f)
    
    transactions = list(reader)
    total = len(transactions)
    
    print(f"Starting processing of {total} transactions...\n")

    for i, row in enumerate(transactions, 1):
        merchant = row['Merchant']
        print(f"[{i}/{total}] Processing: {merchant}...", end=" ", flush=True)
        
        # Call the AI
        result = classify_transaction(row)
        
        # Print results
        if "error" in result:
            print(f"❌ Error: {result['error']}")
        else:
            print("✅ Done")
            print(f"   -> Category: {result.get('category')}")
            print(f"   -> Reasoning: {result.get('reasoning')}")
        
        print("-" * 50)

if __name__ == "__main__":
    main()